# Actual Internet Sports Gambling Activity

---

*by Nam NGUYEN*

This is a revised version of the original notebook that includes more variables, cleaner codes and less clutter.

## Initializing the enviroment/packages/libraries/data

In [64]:
# Initializing
library("haven")
library("dplyr")
library("tidyr")
library("readxl")
library("lubridate")

In [65]:
# Getting data from sas7bdat
demo <- read_sas("RawDataIDemographics.sas7bdat")
pcc <- read_sas("RawDataIIIPokerChipConversions.sas7bdat")
uda <- read_sas("RawDataIIUserDailyAggregation.sas7bdat")

In [66]:
# Getting appendices
file <- "Appendices Group Assignment.xlsx"

product <- read_excel(file, sheet = 1, col_names = c("ProductID", "ProductDesc"), skip = 1)
country <- read_excel(file, sheet = 2, col_names = c("Country", "CountryName"), skip = 1)
lang <- read_excel(file, sheet = 3, col_names = c("Language", "LangDesc"), skip = 1)
app <- read_excel(file, sheet = 4, col_names = c("ApplicationID", "AppDesc"), skip = 1)

In [67]:
# Turning off scientific notation
options(scipen=999)

## Data cleaning

In [68]:
# Imputing missing gender in demo + removing duplicates
demo[is.na(demo$Gender), "Gender"] <- which.max((demo$Gender))
pcc_nodupe <- pcc %>% distinct()
uda_nodupe <- uda %>% distinct()

## Feature engineering

### Demographics

In [69]:
# Cleaning, joining with appendices and data type 
demo[is.na(demo$Gender), "Gender"] <- which.max((demo$Gender))

demo_cleaned <- demo %>%
    left_join(country, by = "Country") %>%
    left_join(lang, by = "Language") %>%
    left_join(app, by = "ApplicationID") %>%
    mutate(RegDate = as.Date(RegDate), 
           FirstPay = ymd(FirstPay, quiet = TRUE),
           FirstAct = ymd(FirstAct, quiet = TRUE),
           Gender = factor(Gender, labels = c("Female", "Male")),
           UserID = as.character(UserID)
          ) %>%
    # Since we will have the first play date for each product and the categorization of these products are not clear, we will be removing the 4 first-day variables
    select(-c(FirstSp, FirstCa, FirstGa, FirstPo, Country, Language, ApplicationID)) %>%
    filter(RegDate %within% interval(ymd("2005-02-01"), ymd("2005-02-27")), !is.na(FirstAct))

### Poker Chip Conversions

In [70]:
# Data type conversion
pcc_cleaned <- pcc_nodupe %>%
    mutate(UserID = as.character(UserID), 
           TransDateTimeDTTM = ymd_hms(TransDateTime), 
           TransType = factor(TransType, labels = c("Buy", "Sell"))) %>%
    select(-TransDateTime)

In [71]:
# Compiling simple statistics
pcc_enriched <- pcc_cleaned %>%
    group_by(UserID) %>%
    summarize(
        NumberOfTimesSelling = sum(TransType == "Sell"),
        NumberOfTimesBuying = sum(TransType == "Buy"),
        TotalSellingAmount = sum((TransType == "Sell") * TransAmount),
        TotalBuyingAmount = sum((TransType == "Buy") * TransAmount),
        AverageBuyOrderValue = TotalBuyingAmount / NumberOfTimesBuying,
        Earnings = TotalSellingAmount - TotalBuyingAmount,
        IsProfiting = ifelse(Earnings > 0, 1, 0),
        FirstTransactionDate = min(TransDateTimeDTTM),
        LastTransactionDate = max(TransDateTimeDTTM),
        LifespanInMonths = floor(as.numeric(FirstTransactionDate %--% LastTransactionDate, "months")) + 1,
        MonthlySpending = TotalBuyingAmount / LifespanInMonths,
        MonthlyOrders = NumberOfTimesBuying / LifespanInMonths,
    )

# Getting buying and selling amount for each month
for (i in min(unique(month(pcc_cleaned$TransDateTimeDTTM))):max(unique(month(pcc_cleaned$TransDateTimeDTTM)))) {
    temp <- pcc_cleaned %>%
        group_by(UserID) %>%
        summarize(
            Buying = sum((month(TransDateTimeDTTM) == i) * (TransType == "Buy")),
            Selling = sum((month(TransDateTimeDTTM) == i) * (TransType == "Sell")),
            BuyingAmount = sum((month(TransDateTimeDTTM) == i) * (TransType == "Buy") * TransAmount),
            SellingAmount = sum((month(TransDateTimeDTTM) == i) * (TransType == "Sell") * TransAmount),
        )
    colnames(temp)[-1] <- lapply(colnames(temp)[-1], function(x) paste(month.abb[i], x, sep = "_"))
    pcc_enriched <- pcc_enriched %>%
        left_join(temp, by = "UserID")
}

colnames(pcc_enriched)[-1] <- lapply(colnames(pcc_enriched)[-1], function(x) paste("Poker", x, sep = "_"))

In [72]:
# Creating a staging table to calculate customer purchasing behaviors
pcc_timediff <- pcc_cleaned %>%
    filter(TransType == "Buy") %>%
    arrange(UserID, TransDateTimeDTTM) %>%
    mutate(
        PokerDateTimeDiff = ifelse(UserID != lag(UserID), NA, c(NA, as.numeric(as.period(diff(TransDateTimeDTTM)), "days"))),
        PokerDateDayDiff = ifelse(UserID != lag(UserID), NA, c(NA, as.numeric(as.period(diff(as_date(TransDateTimeDTTM))), "days"))),
        StreakMarks = cumsum(ifelse((replace(PokerDateDayDiff,is.na(PokerDateDayDiff),999)) <= 2, 0, 1))
    )

pcc_streak <- pcc_timediff %>% count(StreakMarks, name = "StreakLength")

pcc_timediff <- merge(pcc_timediff, pcc_streak, by = "StreakMarks")

In [73]:
# Summarizing the staging table
pcc_timediff <- pcc_timediff %>%
    group_by(UserID) %>%
    summarize(
        NumberOfStreaks = ifelse(n() == n_distinct(StreakMarks), 0, n_distinct(StreakMarks)),
        ShortestStreak = ifelse(NumberOfStreaks == 0, 0, min(StreakLength)),
        LongestStreak = ifelse(NumberOfStreaks == 0, 0, max(StreakLength)),
        ShortestGapBtwOrders = ifelse(n() == 1, NA, min(PokerDateTimeDiff, na.rm = TRUE)),
        LongestGapBtwOrders = ifelse(n() == 1, NA, max(PokerDateTimeDiff, na.rm = TRUE)),
        AverageDaysBtwOrders = ifelse(n() == 1, NA, mean(PokerDateTimeDiff, na.rm = TRUE))
    )

colnames(pcc_timediff)[-1] <- lapply(colnames(pcc_timediff)[-1], function(x) paste("Poker", x, sep = "_"))

In [74]:
# Merging into one
pcc_enriched <- left_join(pcc_enriched, pcc_timediff, by = "UserID")

### User Daily Aggregation

In [75]:
# Joining with appendices, joining with demo to get rid of dates after the first-pay date, filtering out erroneous data 
# and data type conversion
uda_cleaned <- uda_nodupe %>% 
    filter(!(Stakes == 0 & Winnings == 0 & Bets == 0), # Removing 186,435 rows
           !(Stakes < 0 & Winnings < 0 & Bets < 0), # Removing 1 all-negative row
          ) %>%   
    mutate(Winnings = ifelse(Winnings < 0, 0, Winnings), # Replacing negative winnings with 0
           UserID = as.character(UserID), Date = ymd(Date)
          ) %>%
    left_join(select(demo_cleaned, UserID, FirstPay), by = "UserID") %>%
    filter(Date >= FirstPay)

In [76]:
# A function to create variables for each product id to be used later in a loop
create_var <- function(pid) {
    p <- uda_cleaned %>%
        filter(ProductID == pid) %>%
        group_by(UserID) %>%
        summarize(
            FirstPlayDate = min(Date),
            LastPlayDate = max(Date),
            TotalStakes = sum(Stakes),
            TotalWinnings = sum(Winnings),
            TotalBets = sum(Bets),
            TotalProfits = TotalWinnings - TotalStakes,
            IsProfiting = ifelse(TotalProfits > 0, 1, 0),
            LifespanInMonths = floor(as.numeric(FirstPlayDate %--% LastPlayDate, "months")) + 1,
            AvgStakesOverLifespan = TotalStakes / LifespanInMonths,
            AvgStakesPerBet = TotalStakes / TotalBets,
            AvgWinningsOverLifespan = TotalWinnings / LifespanInMonths,
            AvgWinningsPerBet = TotalWinnings / TotalBets,
            AvgBetsOverLifespan = TotalBets / LifespanInMonths
        )
    
    # Creating monthly variables
    for (i in min(unique(month(uda_cleaned$Date))):max(unique(month(uda_cleaned$Date)))) {
        temp <- uda_cleaned %>%
            group_by(UserID) %>%
            summarize(
                Stakes = sum((month(Date) == i) * Stakes),
                Winnings = sum((month(Date) == i) * Winnings),
                Bets = sum((month(Date) == i) * Bets),
                Profits = sum((month(Date) == i) * (Winnings - Stakes)),
            )
        colnames(temp)[-1] <- lapply(colnames(temp)[-1], function(x) paste(month.abb[i], x, sep = "_"))
        p <- p %>%
            left_join(temp, by = "UserID")
    }
    
    colnames(p)[-1] <- lapply(colnames(p)[-1],function(x) paste(paste("BettingProd", pid, sep = ""), x, sep = "_"))
                                                
    return(p)
}

In [77]:
# A function to generate consecutive betting days data for each product ID to be used later in a loop
find_streak <- function(pid) {
    uda_psubset <- uda_cleaned %>%
        filter(ProductID == pid, Bets > 0) %>%
        arrange(UserID, Date) %>%
        mutate(
            DateDiff = ifelse(UserID != lag(UserID), NA, c(NA, as.numeric(as.period(diff(Date)), "days"))),
            StreakMarks = cumsum(ifelse((replace(DateDiff, is.na(DateDiff), 999)) == 1, 0, 1))
        )

    uda_streak <- uda_psubset %>% count(StreakMarks, name = "StreakLength")

    uda_psubset <- uda_psubset %>% 
        merge(uda_streak, by = "StreakMarks") %>%
        group_by(UserID) %>%
        summarize(
            NumberOfStreaks = ifelse(n() == n_distinct(StreakMarks), 0, n_distinct(StreakMarks)),
            ShortestStreak = ifelse(NumberOfStreaks == 0, 0, min(StreakLength)),
            LongestStreak = ifelse(NumberOfStreaks == 0, 0, max(StreakLength)),
            ShortestGapBtwBets = ifelse(n() == 1, NA, min(DateDiff, na.rm = TRUE)),
            LongestGapBtwBets = ifelse(n() == 1, NA, max(DateDiff, na.rm = TRUE)),
            AverageDaysBtwBets = ifelse(n() == 1, NA, mean(DateDiff, na.rm = TRUE))
    )
    
    # Appending the product name to each variable's name to distinguish them
    colnames(uda_psubset)[-1] <- 
        lapply(colnames(uda_psubset)[-1],function(x) paste(paste0("BettingProd", pid), x, sep = "_"))
    
    return(uda_psubset)
}

In [78]:
# A loop to run through all product ids
uda_enriched <- uda_cleaned %>%
    select(UserID) %>%
    distinct()

for (i in 1:max(uda_cleaned$ProductID)) {
    if (i == 3) next # No poker product in this table
    temp1 <- create_var(i)
    temp2 <- find_streak(i)
    
    uda_enriched <- uda_enriched %>%
        left_join(temp1, by = "UserID") %>%
        left_join(temp2, by = "UserID")
}

In [79]:
# Generating the most played betting product for one customer
uda_temp <- uda_enriched %>% select(ends_with("_TotalBets"))

uda_enriched <- uda_enriched %>%
    mutate(FavoriteBettingProduct = apply(uda_temp, 1, which.max),
           # Because there is no Poker (id = 3)
           FavoriteBettingProduct = ifelse(FavoriteBettingProduct > 3, FavoriteBettingProduct + 1, FavoriteBettingProduct))

## Putting everything together

In [80]:
# Joining all tables together
ult_table <- demo_cleaned %>%
    left_join(pcc_enriched, by = "UserID") %>%
    left_join(uda_enriched, by = "UserID")

In [81]:
# Updating First Active Date, which is correct most of the time, except for customers whose first pay-in date is later than
# the original FirstAct variable
ult_temp <- ult_table %>%
    select(ends_with("_FirstPlayDate"), Poker_FirstTransactionDate) %>%
    mutate(Poker_FirstTransactionDate = as_date((Poker_FirstTransactionDate)))

ult_table <- ult_table %>%
    mutate(FirstActiveDate = suppressWarnings(ymd(apply(ult_temp, 1, min, na.rm = TRUE)))) %>%
    select(-FirstAct)

In [82]:
# Getting last active date
ult_temp <- ult_table %>%
    select(ends_with("_LastPlayDate"), Poker_LastTransactionDate) %>%
    mutate(Poker_LastTransactionDate = as.Date(Poker_LastTransactionDate))

ult_table <- ult_table %>%
    mutate(
        LastActiveDate = suppressWarnings(ymd(apply(ult_temp, 1, max, na.rm = TRUE))),
    )

In [83]:
# Getting total spending
ult_temp <- ult_table %>%
    select(ends_with("_TotalStakes"), Poker_TotalBuyingAmount)

ult_table <- ult_table %>%
    mutate(
        TotalSpending = apply(ult_temp, 1, sum, na.rm = TRUE)
    )

In [84]:
# Segmenting customers based on poker chips buying amount
ult_table <- ult_table %>%
    mutate(
        Segment = cut(
            TotalSpending, 
            breaks = quantile(ult_table$TotalSpending, 0:4/4), 
            labels = c("Small Spender", "Medium Spender", "Big Spender", "Whale")
        )
    )

In [85]:
# Write to file
write.csv(ult_table, "basetable.csv")